In [9]:
#Create Files_Name
image_data='C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
pd.DataFrame(os.listdir(image_data),columns=['Files_Name'])

,Files_Name
0,Bird-drop
1,Clean
2,Dusty
3,Electrical-damage
4,Physical-Damage
5,Snow-Covered


In [15]:

from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b0'  # b5

image_size = EfficientNet.get_image_size(model_name)
print(image_size)
model = EfficientNet.from_pretrained(model_name, num_classes=6)

224
Loaded pretrained weights for efficientnet-b0


In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=5)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


## efficientnet b0 학습

In [12]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=10)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
cuda
Epoch 1/50
Train Loss: 0.9612, Train Accuracy: 0.6935
Test Loss: 1.8026, Test Accuracy: 0.5706
Epoch 2/50
Train Loss: 0.3584, Train Accuracy: 0.8884
Test Loss: 1.3889, Test Accuracy: 0.7175
Epoch 3/50
Train Loss: 0.1668, Train Accuracy: 0.9534
Test Loss: 1.1210, Test Accuracy: 0.7740
Epoch 4/50
Train Loss: 0.1748, Train Accuracy: 0.9534
Test Loss: 0.9343, Test Accuracy: 0.8305
Epoch 5/50
Train Loss: 0.1335, Train Accuracy: 0.9520
Test Loss: 0.5942, Test Accuracy: 0.8588
Epoch 6/50
Train Loss: 0.0912, Train Accuracy: 0.9732
Test Loss: 0.8182, Test Accuracy: 0.8531
Epoch 7/50
Train Loss: 0.1392, Train Accuracy: 0.9619
Test Loss: 0.7076, Test Accuracy: 0.8757
Epoch 8/50
Train Loss: 0.1489, Train Accuracy: 0.9675
Test Loss: 0.6894, Test Accuracy: 0.8757
Epoch 9/50
Train Loss: 0.0894, Train Accuracy: 0.9689
Test Loss: 0.8663, Test Accuracy: 0.8305
Epoch 10/50
Train Loss: 0.0725, Train Accuracy: 0.9831
Test Loss: 1.1652, Test Accuracy: 0.830

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ 모델 정의
class ModifiedEfficientNet(nn.Module):
    def __init__(self, num_classes=6):
        super(ModifiedEfficientNet, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()  # 기존 Fully Connected Layer 제거

        # 새롭게 정의한 Classifier 추가
        self.classifier = nn.Sequential(
            nn.Linear(1280, 512),  # EfficientNet-b0의 출력 feature 수는 1280
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.efficientnet.extract_features(x)  # EfficientNet에서 특징 추출
        features = nn.functional.adaptive_avg_pool2d(features, 1).view(features.size(0), -1)  # GAP 적용
        return self.classifier(features)
# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedEfficientNet(num_classes=6).to(device)  # 수정된 EfficientNet 적용

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")

Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.0735 | Train Acc: 0.6299 | Val Loss: 2.5013 | Val Acc: 0.4972
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.3799 | Train Acc: 0.8842 | Val Loss: 1.5851 | Val Acc: 0.6780
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.2491 | Train Acc: 0.9153 | Val Loss: 2.1996 | Val Acc: 0.7684
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.1733 | Train Acc: 0.9492 | Val Loss: 1.1097 | Val Acc: 0.7797
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.0608 | Train Acc: 0.9845 | Val Loss: 0.8766 | Val Acc: 0.8475
  [*] Best model saved.

[Epoch 18/100]


Train Loss: 0.1551 | Train Acc: 0.9661 | Val Loss: 1.5219 | Val Acc: 0.7627

[Epoch 19/100]


Train Loss: 0.0844 | Train Acc: 0.9760 | Val Loss: 0.7975 | Val Acc: 0.8814
  [*] Best model saved.

[Epoch 20/100]


Train Loss: 0.1698 | Train Acc: 0.9449 | Val Loss: 1.3413 | Val Acc: 0.7853

[Epoch 21/100]


Train Loss: 0.1228 | Train Acc: 0.9647 | Val Loss: 1.0069 | Val Acc: 0.8079
Early Stopping Triggered!

Best Accuracy: 0.8814


In [14]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 16
epochs = 50
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=10)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
cuda
Epoch 1/50
Train Loss: 0.9148, Train Accuracy: 0.7090
Test Loss: 0.7648, Test Accuracy: 0.7514
Epoch 2/50
Train Loss: 0.4759, Train Accuracy: 0.8503
Test Loss: 0.9004, Test Accuracy: 0.7853
Epoch 3/50
Train Loss: 0.2824, Train Accuracy: 0.9082
Test Loss: 0.8215, Test Accuracy: 0.7853
Epoch 4/50
Train Loss: 0.3222, Train Accuracy: 0.8884
Test Loss: 1.1597, Test Accuracy: 0.7571
Epoch 5/50
Train Loss: 0.1962, Train Accuracy: 0.9251
Test Loss: 0.4637, Test Accuracy: 0.8927
Epoch 6/50
Train Loss: 0.1601, Train Accuracy: 0.9520
Test Loss: 0.9085, Test Accuracy: 0.7684
Epoch 7/50
Train Loss: 0.1847, Train Accuracy: 0.9421
Test Loss: 0.6036, Test Accuracy: 0.8814
Epoch 8/50
Train Loss: 0.1401, Train Accuracy: 0.9506
Test Loss: 0.5171, Test Accuracy: 0.8757
Epoch 9/50
Train Loss: 0.1040, Train Accuracy: 0.9661
Test Loss: 0.4718, Test Accuracy: 0.8644
Epoch 10/50
Train Loss: 0.1499, Train Accuracy: 0.9534
Test Loss: 0.8708, Test Accuracy: 0.841

In [13]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label Smoothing 적용
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=10)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
cuda
Epoch 1/50
Train Loss: 1.1301, Train Accuracy: 0.6907
Test Loss: 1.4589, Test Accuracy: 0.5932
Epoch 2/50
Train Loss: 0.6919, Train Accuracy: 0.8983
Test Loss: 1.0938, Test Accuracy: 0.7571
Epoch 3/50
Train Loss: 0.5606, Train Accuracy: 0.9590
Test Loss: 0.8848, Test Accuracy: 0.8192
Epoch 4/50
Train Loss: 0.5216, Train Accuracy: 0.9760
Test Loss: 0.7556, Test Accuracy: 0.8757
Epoch 5/50
Train Loss: 0.4940, Train Accuracy: 0.9845
Test Loss: 0.6625, Test Accuracy: 0.9153
Epoch 6/50
Train Loss: 0.4726, Train Accuracy: 0.9915
Test Loss: 0.7083, Test Accuracy: 0.8927
Epoch 7/50
Train Loss: 0.4725, Train Accuracy: 0.9887
Test Loss: 0.6988, Test Accuracy: 0.9040
Epoch 8/50
Train Loss: 0.5074, Train Accuracy: 0.9718
Test Loss: 0.8252, Test Accuracy: 0.8531
Epoch 9/50
Train Loss: 0.4796, Train Accuracy: 0.9915
Test Loss: 0.7390, Test Accuracy: 0.8983
Epoch 10/50
Train Loss: 0.4756, Train Accuracy: 0.9915
Test Loss: 0.7728, Test Accuracy: 0.853

## efficientnet b1 학습

In [5]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b1'
num_classes = 6
batch_size = 32
epochs = 20
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=5)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to C:\Users\IIALAB/.cache\torch\hub\checkpoints\efficientnet-b1-f1951068.pth
100.0%


Loaded pretrained weights for efficientnet-b1
cuda
Epoch 1/20
Train Loss: 1.0324, Train Accuracy: 0.6610
Test Loss: 1.4352, Test Accuracy: 0.4859
Epoch 2/20
Train Loss: 0.4420, Train Accuracy: 0.8729
Test Loss: 0.7149, Test Accuracy: 0.8079
Epoch 3/20
Train Loss: 0.2560, Train Accuracy: 0.9223
Test Loss: 0.8063, Test Accuracy: 0.7966
Epoch 4/20
Train Loss: 0.1418, Train Accuracy: 0.9605
Test Loss: 0.9152, Test Accuracy: 0.8023
Epoch 5/20
Train Loss: 0.1275, Train Accuracy: 0.9605
Test Loss: 0.7893, Test Accuracy: 0.8418
Epoch 6/20
Train Loss: 0.1284, Train Accuracy: 0.9562
Test Loss: 0.9402, Test Accuracy: 0.8079
Early stopping triggered

Training complete! Best Test Accuracy: 0.8418 at Epoch 5


## efficientnet b2 학습

In [6]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b2'
num_classes = 6
batch_size = 32
epochs = 20
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=5)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to C:\Users\IIALAB/.cache\torch\hub\checkpoints\efficientnet-b2-8bb594d6.pth
100.0%


Loaded pretrained weights for efficientnet-b2
cuda
Epoch 1/20
Train Loss: 0.9927, Train Accuracy: 0.6737
Test Loss: 2.8050, Test Accuracy: 0.5819
Epoch 2/20
Train Loss: 0.4010, Train Accuracy: 0.8842
Test Loss: 0.8842, Test Accuracy: 0.7627
Epoch 3/20
Train Loss: 0.1944, Train Accuracy: 0.9421
Test Loss: 1.5038, Test Accuracy: 0.6780
Epoch 4/20
Train Loss: 0.1956, Train Accuracy: 0.9421
Test Loss: 1.1412, Test Accuracy: 0.8023
Epoch 5/20
Train Loss: 0.1554, Train Accuracy: 0.9534
Test Loss: 1.4277, Test Accuracy: 0.7514
Epoch 6/20
Train Loss: 0.1989, Train Accuracy: 0.9421
Test Loss: 0.7379, Test Accuracy: 0.8531
Epoch 7/20
Train Loss: 0.1289, Train Accuracy: 0.9633
Test Loss: 0.7506, Test Accuracy: 0.8418
Epoch 8/20
Train Loss: 0.2375, Train Accuracy: 0.9336
Test Loss: 1.2633, Test Accuracy: 0.7345
Epoch 9/20
Train Loss: 0.1888, Train Accuracy: 0.9520
Test Loss: 1.4378, Test Accuracy: 0.7514
Epoch 10/20
Train Loss: 0.1556, Train Accuracy: 0.9548
Test Loss: 1.3659, Test Accuracy: 0.774

## efficientnet b5 학습

In [10]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False


# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b5'
num_classes = 6
batch_size = 32
epochs = 20
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
best_test_acc = 0.0  # 최고 Test Accuracy 저장
best_epoch = 0       # 최고 Test Accuracy를 달성한 Epoch
early_stopping = EarlyStopping(patience=5)  # Early Stopping 설정

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    # 최고 Test Accuracy 갱신
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    # Early Stopping 체크
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Epoch 결과 출력
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# 최고 Test Accuracy 출력
print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b5
cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 306.00 MiB (GPU 0; 19.99 GiB total capacity; 17.85 GiB already allocated; 0 bytes free; 19.06 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF